Julia has a few well designed interfaces, including **arrays** and **iterators**.

# Arrays

## Built-in arrays

Just like Fortran, Julia has really good support for multi-dimensional arrays (1-based by default). Julia has a default implementation of arrays that use **column-major order**.

For small arrays, a convenient syntax uses semi-columns to separate the elements along any given number of dimensions (`;` for the first, `;;` for the second...).

In [1]:
a = [ 1;  2;  3;  4;;  5;  6;  7;  8;;  9;  10;  11;  12;;;
     -1; -2; -3; -4;; -5; -6; -7; -8;; -9; -10; -11; -12]

4×3×2 Array{Int64, 3}:
[:, :, 1] =
 1  5   9
 2  6  10
 3  7  11
 4  8  12

[:, :, 2] =
 -1  -5   -9
 -2  -6  -10
 -3  -7  -11
 -4  -8  -12

The brackets notation is used to read (`getindex`) or write (`setindex!`) an element:

In [2]:
a[4, 1, 2]

-4

In [3]:
a[4, 1, 2] = 100
a

4×3×2 Array{Int64, 3}:
[:, :, 1] =
 1  5   9
 2  6  10
 3  7  11
 4  8  12

[:, :, 2] =
  -1  -5   -9
  -2  -6  -10
  -3  -7  -11
 100  -8  -12

## Custom `AbstractArray`s

Anything that inherents from `AbstractArray{T,N}` (a `N`-dimensional array of elements of type `T`) will be able to use default implementations of various algorithms (`reduce`, `mapreduce`...) provided a minimal set of methods are added to `Base` functions:

- `Base.size`,
- `Base.getindex`,
- (`Base.setindex!` for mutable arrays).

Let's assume you want to create our own `MetaArray` which wraps another array together with some metadata (the position of a field, *i.e.* cell-centered, face-centered, ..., the name of a variable, etc.).

In [4]:
struct MetaArray{T,N,A<:AbstractArray{T,N},D} <: AbstractArray{T,N}
    data::A
    meta::D
end

This is an **immutable** type (to make **mutable**, simply prepend `mutable` to `struct`). So we only need define a couple of getter functions.

In [5]:
Base.parent(this::MetaArray) = this.data
metadata(this::MetaArray) = this.meta

metadata (generic function with 1 method)

Adding the methods listed above is done below.

In [6]:
Base.size(this::MetaArray) = size(parent(this))
Base.getindex(this::MetaArray{T,N}, ind::Vararg{Int,N}) where {T,N} =
    getindex(parent(this), ind...)
Base.setindex!(this::MetaArray{T,N}, val, ind::Vararg{Int,N}) where {T,N} =
    setindex!(parent(this), val, ind...)

It is also possible to work with 0-based (or anything really, just like in Fortran), a method for `Base.axes` also needs to be added.

<div class="alert alert-block alert-info">
    <b>Info</b>: a good practice in Julia is to append a <i>bang</i> (<tt>!</tt>) to functions that mutate their arguments (the first one, usually), as in <tt>setindex!</tt>.
</div>

Let's now wrap a 3x2 array of random `Float64` numbers with a string:

In [7]:
ma = MetaArray(rand(3, 2), "My first MetaArray")

3×2 MetaArray{Float64, 2, Matrix{Float64}, String}:
 0.0802089  0.512063
 0.565775   0.125948
 0.179825   0.79424

We can now use a lot of higher-level functionnalities:

In [8]:
minimum(ma), maximum(ma), sum(ma), sum(sin, ma)

(0.0802089382165393, 0.7942397006437546, 2.2580588940685855, 2.1239727701546838)

Any given function could also be applied element-wise (using a `.` as in Matlab) to our `MetaArray`.

In [9]:
exp.(ma)

3×2 Matrix{Float64}:
 1.08351  1.66873
 1.76081  1.13422
 1.19701  2.21276

As seen above, the metadata was lost in the process since the returned object is a `<:Matrix` (Julia's default 2D implementation). This can be fixed by customizing the **broadcasting** machinery.

## Another example: `LazyArray`

Let's assume you want to represent the lazy representation of a function applied to an array. This can also be easily done.


In [10]:
struct LazyArray{T,N,S,A<:AbstractArray{S,N},F} <: AbstractArray{T,N}
    data::A
    f::F
end

Base.parent(this::LazyArray) = this.data
handle(this::LazyArray) = this.f

Base.size(this::LazyArray) = size(parent(this))
Base.getindex(this::LazyArray{T,N}, ind::Vararg{Int,N}) where {T,N} =
    handle(this)(getindex(parent(this), ind...))

To infer the element type of a `LazyArray` (that is, the return type of `f` applied to an element of `data`), we also need to define a custom constructor.

In [11]:
function LazyArray(data::A, f::F) where {S,N,A<:AbstractArray{S,N},F}
    T = Base._return_type(f, Tuple{S})
    LazyArray{T,N,S,A,F}(data, f)
end

LazyArray

For a lazy representation of the exponential:

In [12]:
LazyArray(rand(2, 3), exp)

2×3 LazyArray{Float64, 2, Float64, Matrix{Float64}, typeof(exp)}:
 1.57413  2.07722  2.5268
 1.54051  1.33129  1.40943

But anything else works too:

In [13]:
myfun(x) = (x, x ^ 2)

myfun (generic function with 1 method)

In [14]:
LazyArray(rand(2, 3), myfun)

2×3 LazyArray{Tuple{Float64, Float64}, 2, Float64, Matrix{Float64}, typeof(myfun)}:
 (0.695772, 0.484099)  (0.435251, 0.189443)  (0.0475702, 0.00226292)
 (0.860136, 0.739833)  (0.957631, 0.917058)  (0.629729, 0.396559)

To illustrate the point of traits, let's say you know beforehand you want to compute the extrema after appling expensive function (such as `exp`) to an array. 

In [15]:
abstract type MonotonicityStyle end

struct Increasing <: MonotonicityStyle end
struct Decreasing <: MonotonicityStyle end
struct Unknown <: MonotonicityStyle end

# default
MonotonicityStyle(f) = Unknown()

MonotonicityStyle

Since we know `exp` to be monotonically increasing, we just have to say it.

In [16]:
MonotonicityStyle(::typeof(exp)) = Increasing()

MonotonicityStyle

In [17]:
Base.maximum(this::LazyArray) =
    _maximum(MonotonicityStyle(handle(this)), handle(this), parent(this))

_maximum(::Increasing, f::Function, data::AbstractArray) =
    f(maximum(data))

_maximum (generic function with 1 method)

To compare the cost between the two, we run into the issue that because Julia uses Just-in-Time (or Ahead-of-Time) compilation (based on the [LLVM](https://llvm.org/) compiler infrastructure), the time take by a function call **first time it is run** also includes the compilation time.

As a result, the function is run a first time before timing it in the cell below.

In [ ]:
a = rand(1000, 1000)
la = LazyArray(a, exp)

maximum(la); @time maximum(la)
maximum(exp.(a)); @time maximum(exp.(a))

## Iterators

Julia also has a very simple iterator interface. Let's for example implement the `Fibonacci` sequence as an iterator.

In [ ]:
struct Fibonacci
    len::Int
end

Base.length(this::Fibonacci) = this.len

function Base.iterate(this::Fibonacci, state=(0, 0, 1))
    i, prev, next = state
    i ≥ length(this) && return nothing
    next, (i+1, next, prev+next)
end

The `iterate` method above includes a short-circuit "and" operator (the expression to its right is not evaluated unless the first is `true`). Depending on the value of `i ≥ length(this)`, the return value can be either `Nothing`, or `Tuple{Int,NTuple{3,Int}}`. As seen below by inspecting the code using the `@code_warntype`, the returned type is therefore a `Union{Nothing, Tuple{Int64, Tuple{Int64, Int64, Int64}}}`, which is called a `Union` type.

In [ ]:
@code_warntype iterate(Fibonacci(1), (2, 0, 0))

In [ ]:
for el in Fibonacci(10)
    @show el
end

To look at the assembly code generated, we can include a loop in a function and sum the first `len` elements of the Fibonacci sequence.

In [ ]:
function fibosum(len)
    iter = Fibonacci(len)
    n = 0
    for el in iter
        n += el
    end
    n
end

In [ ]:
@code_native fibosum(10)

# Available implementations

There exists a wide range of custom arrays in Julia:

1. From the [Julia standard library](https://docs.julialang.org/en/v1/stdlib),

    - [SharedArrays](https://docs.julialang.org/en/v1/stdlib/SharedArrays/),
    - [SparseArrays](https://docs.julialang.org/en/v1/stdlib/SparseArrays/),

1. From the [JuliaArrays](https://github.com/JuliaArrays) organisation,

    - [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl),
    - [OffsetArrays.jl](https://github.com/JuliaArrays/OffsetArrays.jl),
    
1. From the [JuliaGPU](https://github.com/JuliaGPU) organisation, [GPUArrays.jl](https://github.com/JuliaGPU/GPUArrays.jl) and used by

    - [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl),
    - [oneAPI.jl](https://github.com/JuliaGPU/oneAPI.jl),
    - [AMDGPU.jl](https://github.com/JuliaGPU/AMDGPU.jl),
    - [Metal.jl](https://github.com/JuliaGPU/Metal.jl).